Необходимые импорты

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import Adam

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score

import re
import pandas as pd

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers.onnx import FeaturesManager

from prettytable import PrettyTable

c:\Users\SsaWin\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Импорт класса для обработки текста

In [2]:
from text_preproccessor import TextPreproccessor

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SsaWin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Структура данных для датасета

In [3]:
class DataSet(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer_path="cointegrated/rubert-tiny2"):
        super(DataSet, self).__init__()
        
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

        self.texts = texts
        self.labels = labels

        self.text_preprocessor = TextPreproccessor()

    def tokenize(self, text: str):
        t = self.tokenizer(text, padding='max_length', truncation=True, return_tensors='pt')

        input_ids = t['input_ids']
        token_type_ids = t['token_type_ids']
        attention_mask = t['attention_mask']
        
        return input_ids.squeeze(), token_type_ids.squeeze(), attention_mask.squeeze()

    def __getitem__(self, idx: int):
        text = self.texts[idx]
        label = self.labels[idx]

        text = self.text_preprocessor.preproccess_text(text)

        input_ids, token_type_ids, attention_mask = self.tokenize(text)

        return input_ids, token_type_ids, attention_mask, label
    
    def __len__(self):
        return len(self.texts)



TinyBert Классификатор

In [4]:
class TinyBertClassificator(nn.Module):
    def __init__(self, num_labels=1, dropout=0.25, embedding_size=312, model="cointegrated/rubert-tiny2"):
        super(TinyBertClassificator, self).__init__()

        self.bert_model = AutoModelForSequenceClassification.from_pretrained(model, num_labels=num_labels)
        
        self.dropout = nn.Dropout(dropout)

        self.linear = nn.Linear(embedding_size, num_labels)

        self.softmax = nn.Softmax()

        self.reLU = nn.ReLU()
    
    def forward(self, input_ids, token_type_ids, attention_mask):
        output = self.bert_model(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)[0]

        return output


Обучение сети

In [5]:
def train(model: TinyBertClassificator, train_dataloader: DataLoader, loss_f, optimizer, batch_size=4, train=True):
    model.train(train)

    train_loss = 0

    for i, batch in enumerate(tqdm(train_dataloader)):
        input_ids, token_type_ids, attention_mask, labels = (t.cpu() for t in batch)

        output = model(input_ids, token_type_ids, attention_mask)

        batch_loss = loss_f(output, labels)

        if train:
            optimizer.zero_grad()  # обнуляем градиенты
            batch_loss.backward()  # вычисляем градиенты
            optimizer.step()  # подправляем параметры

        train_loss = batch_loss.item()
    
    return train_loss / len(train_dataloader)


def test(model: TinyBertClassificator, test_dataloader: DataLoader, loss_f, batch_size=4):
    total_loss_test = 0
    predicts = []
    y_true = []

    with torch.no_grad():
        for batch in tqdm(test_dataloader):
            input_ids, token_type_ids, attention_mask, labels = (t.cpu() for t in batch)

            output = model(input_ids, token_type_ids, attention_mask)

            predicts.extend(output.argmax(-1).tolist())
            y_true.extend(labels.tolist())

            batch_loss = loss_f(output, labels)
            total_loss_test += batch_loss.item()

    total_loss_test = total_loss_test / len(test_dataloader)

    accuracy = accuracy_score(y_true, predicts)
    balanced_accuracy = balanced_accuracy_score(y_true, predicts)
    f1 = f1_score(y_true, predicts, average='weighted')

    metrics = {"Accuracy": accuracy,
            "Balanced_accuracy": balanced_accuracy,
            "F1-score": f1,
            "Test loss": total_loss_test }

    return metrics


In [6]:
def get_texts_from_excel_file(file_path: str) -> pd.DataFrame:
    return pd.read_excel(file_path).dropna()

Получение данных из файла

In [18]:
TEXTS_FILEPATH = 'texts.xlsx'

texts_df = get_texts_from_excel_file(TEXTS_FILEPATH)


Подготовка данных

In [19]:
class_counts = texts_df['name'].value_counts()

#Считаем встречаемость классов в датасете
name2count = {name:count for name,count in zip(class_counts.index, class_counts.values)}

In [20]:
#Добавляем в табличку столбец вхождений класса
texts_df['class_count'] = texts_df['name'].apply(lambda x: name2count[x])

In [21]:
data = texts_df[texts_df['class_count'] > 1][['description', 'name']]

In [22]:
name2class_id = {name: idx for idx, name in enumerate(data['name'].unique())}
#Добавляем в табличку столбец с индексами классов
data['class_id'] = data['name'].apply(lambda x: name2class_id[x])
data.drop(['name'], axis=1, inplace=True)

127


In [16]:
X = data['description'].values
y = data['class_id'].values
#Делим датасет на тренировочную и тестовую выборки, учитывая распределение классов
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [17]:
batch_size = 4                              #Размер батча
epochs = 40                                 #Итерации обучения
n_classes = len(data['class_id'].unique())  #Количество классов в датасете

In [18]:
#Создаем объекты тренировочного и тестового датасетов
train_dataset = DataSet(X_train, y_train)
test_dataset = DataSet(X_test, y_test)

#Оборачиваем в dataloader-ы
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size)


In [21]:
model = TinyBertClassificator(n_classes).cpu()
criterion = nn.CrossEntropyLoss().cpu()
optimizer = Adam(model.parameters(), lr=0.5)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\SsaWin\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\import_utils.py", line 1110, in _get_module
  File "c:\Users\SsaWin\AppData\Local\Programs\Python\Python310\lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1004, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'transformers.models.ernie.configuration_ernie'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\SsaWin\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SsaWin\AppData\Local\Temp\ipykern

In [20]:
table = PrettyTable(["Accuracy", "Balanced_accuracy", "F1-score", "Test loss"])

for epoch in range(epochs):
    train_loss = train(model, train_dataloader, criterion,
                       optimizer, batch_size=batch_size)
    metrics = test(model, test_dataloader, criterion, batch_size=batch_size)
    
    table.add_row([metrics["Accuracy"], metrics["Balanced_accuracy"], metrics["F1-score"], metrics["Test loss"]])

print(table)

NameError: name 'model' is not defined

In [ ]:
table

Accuracy,Balanced_accuracy,F1-score,Test loss


In [ ]:
print(torch.cuda.is_available())

False
